## 5.1 순서가 없는 범주형 특성 인코딩하기
사이킷런의 LabelBinarizer를 사용하여 특성을 원-핫 인코딩합니다.

In [1]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

feature = np.array([["Texas"],
                    ["California"],
                    ["Texas"],
                    ["Delaware"], 
                    ["Texas"]])

one_hot = LabelBinarizer()

one_hot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [2]:
# classes_ 속성에서 클래스를 확인할 수 있습니다.

one_hot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

In [3]:
# 원-핫 인코딩을 되돌리려면 inverse_transform 메서드를 사용합니다.

one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

In [4]:
# 판다스를 사용해서 특성을 원-핫 인코딩할 수도 있습니다.

import pandas as pd

pd.get_dummies(feature[:,0])

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


In [7]:
# 사이킷런에 있는 한 가지 유용한 기능은 샘플이 여러 개의 클래스를 가지고 있는 경우 다룰 수 있다는 것입니다.

multiclass_feature = [("Texas", "Florida"),
                      ("California", "Alabama"),
                      ("Texas", "Florida"),
                      ("Delware", "Florida"),
                      ("Texas", "Alabama")]

one_hot_multiclass = MultiLabelBinarizer()

one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [8]:
# 여기서도 classes_ 속성으로 클래스를 확인할 수 있습니다.

one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delware', 'Florida', 'Texas'],
      dtype=object)

특성을 원-핫 인코딩한 후에는 선형 의존성을 피하기 위해 결과 행렬에서 원-핫 인코딩된 특성 중 하나를 삭제하는 것이 좋습니다.

LabelBinarizer는 문자열 타깃 데이터를 원-핫 인코딩으로 변환할 때 사용합니다. 비슷하게 문자열 타깃 데이터를 정수 레이블로 변환할 때는 LabelEncoder를 사용합니다. 이 두 클래스는 일차원 배열을 기대합니다. 

이전에는 사이킷런의 OneHotEncoder 클래스가 정수형 특성만 원-핫 인코딩으로 변환했지만 0.20 버전부터는 문자열 데이터를 인식할 수 있습니다. 특성 배열을 원-핫 인코딩할 때는 OneHotEncoder 클래스를 사용하는 것이 좋습니다.

OneHotEncoder 클래스는 기본적으로 희소 배열을 반환합니다. sparse_output=False로 지정하면 밀집 배열을 얻을 수 있습니다.

In [12]:
from sklearn.preprocessing import OneHotEncoder

feature = np.array([["Texas", 1],
                    ["California", 1],
                    ["Texas", 3],
                    ["Delaware", 1],
                    ["Texas", 1]])

one_hot_encoder = OneHotEncoder(sparse_output=False)
one_hot_encoder.fit_transform(feature)

array([[0., 0., 1., 1., 0.],
       [1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1.],
       [0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 0.]])

California, Delaware, Texas가 처음 세 개의 열에 원-핫 인코딩되었고 1과 3이 나머지 두 개의 열에 원-핫 인코딩되었습니다. 이 예에서처럼 정수도 문자열처럼 취급하여 변환됩니다.

In [14]:
# categories_ 속성으로 클래스를 확인할 수 있습니다.

one_hot_encoder.categories_

[array(['California', 'Delaware', 'Texas'], dtype='<U10'),
 array(['1', '3'], dtype='<U10')]

OneHotEncoder는 입력 특성 배열을 모두 범주형으로 인식하여 변환합니다. 특정 열에만 적용하려면 이전 장에서 설명한 ColumnTransformer와 함께 사용하세요.

## 5.2 순서가 있는 범주형 특성 인코딩하기
판다스 데이터프레임의 replace 메서드를 사용하여 문자열 레이블을 수치값으로 변환합니다.

In [16]:
import pandas as pd

dataframe = pd.DataFrame({"Score":["Low", "Low", "Medium", "Medium", "High"]})

# 매핑 딕셔너리를 만듭니다.
scale_mapper = {"Low":1,
                "Medium":2,
                "High":3}

# 특성을 정수로 변환합니다.
dataframe["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

어떤 수치값을 선택하는지는 클래스에 내재된 순서 정보에 기반한다는 사실을 유념하세요. 해결에서 high는 글자 그대로 low보다 세 배 더 큽니다. 경우에 따라 이런 설정이 잘 맞을 수 있지만 클래스 사이 간격이 동일하지 않는다면 문제가 됩니다.

In [18]:
dataframe = pd.DataFrame({"Score": ["Low",
                                    "Low",
                                    "Medium",
                                    "Medium",
                                    "High",
                                    "Barely More Than Medium"]})

scale_mapper = {"Low":1,
                "Medium":2,
                "Barely More Than Medium":3,
                "High":4}

dataframe["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    4
5    3
Name: Score, dtype: int64

이 예에서는 Low와 Medium 사이의 거리가 Medium과 Barely More Than Medium 사이의 거리와 같지만 실제로는 그렇지 않습니다. 가장 좋은 방법은 클래스에 매핑하는 수치값에 주의를 기울이는 것입니다.

In [19]:
scale_mapper = {"Low":1,
                "Medium":2,
                "Barely More Than Medium":2.1,
                "High":3}

dataframe["Score"].replace(scale_mapper)

0    1.0
1    1.0
2    2.0
3    2.0
4    3.0
5    2.1
Name: Score, dtype: float64

사이킷런 0.20 버전에 범부형 데이터를 정수로 인코딩하는 OrdinalEncoder가 추가되었습니다. OrdinalEncoder는 클래스 범주를 순서대로 변환합니다. 정수 데이터도 범주형으로 인식하여 변환합니다. 특정 열만 범주형으로 변환하려면 ColumnTransformer와 함께 사용하세요.

In [20]:
from sklearn.preprocessing import OrdinalEncoder

features = np.array([["Low", 10],
                     ["High", 50],
                     ["Medium", 3]])

ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit_transform(features)

array([[1., 0.],
       [0., 2.],
       [2., 1.]])

In [22]:
# 클래스는 categories_ 속성에서 확인할 수 있습니다.

ordinal_encoder.categories_

[array(['High', 'Low', 'Medium'], dtype='<U6'),
 array(['10', '3', '50'], dtype='<U6')]

## 5.3 특성 딕셔너리를 인코딩하기
DictVectorizer를 사용하세요

In [23]:
from sklearn.feature_extraction import DictVectorizer

data_dict = [{"Red": 2, "Blue": 4},
             {"Red": 4, "Blue": 3},
             {"Red": 1, "Yellow": 2},
             {"Red": 2, "Yellow": 2}]
dictvectorizer = DictVectorizer(sparse=False)

features = dictvectorizer.fit_transform(data_dict)

features

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

기본적으로 DictVectorizer는 0이 아닌 값의 원소만 저장하는 희소 행렬을 반환합니다. 이는 매우 큰 행렬을 다루어야 할 때 도움이 됩니다. 메모리 사용량을 최소화해야 하기 때문입니다. DictVectorizer를 sparse=False로 지정하면 밀집 벡터를 출력할 수 있습니다.

In [25]:
# get_feature_names_out 메서드를 사용하여 생성된 특성의 이름을 얻을 수 있습니다.

feature_names = dictvectorizer.get_feature_names_out()

feature_names

array(['Blue', 'Red', 'Yellow'], dtype=object)

In [26]:
# 필수적이진 않지만 미려한 출력을 위해 판다스 데이터프레임으로 출력할 수 있습니다.

import pandas as pd

pd.DataFrame(features, columns=feature_names)

,Blue,Red,Yellow
0,4.0,2.0,0.0
1,3.0,4.0,0.0
2,0.0,1.0,2.0
3,0.0,2.0,2.0


딕셔너리는 많은 프로그래밍 언어에서 즐겨 사용하는 데이터 구조입니다. 하지만 머신러닝 알고리즘은 행렬 형태의 데이터를 기대합니다. 이를 위해 사이킷런의 DictVectorizer를 사용할 수 있습니다.

이런 작업은 자연어 처리 분야에서는 자주 발생하는 일입니다. 예를 들어 문서 데이터를 가지고 있을 때 각 문서에 등장한 모든 단어의 횟수를 담은 딕셔너리를 만들 수 있습니다. dictvectorizer를 사용하면 각 문서에 등장한 단어 횟수를 특성으로 하는 특성 행렬을 만들 수 있습니다.

In [28]:
# 네 개의 문서에 대한 단어 카운트 딕셔너리를 만듭니다.
doc_1_word_count = {"Red": 2, "Blue": 4}
doc_2_word_count = {"Red": 4, "Blue": 3}
doc_3_word_count = {"Red": 1, "Yellow": 2}
doc_4_word_count = {"Red": 2, "Yellow": 2}

# 리스트를 만듭니다.
doc_word_counts = [doc_1_word_count,
                   doc_2_word_count,
                   doc_3_word_count,
                   doc_4_word_count]

# 단어 카운트 딕셔너리를 특성 행렬로 변환합니다.
dictvectorizer.fit_transform(doc_word_counts)

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

이 예제는 고유한 단어가 세 개뿐이므로 행렬에 세 개의 특성만 있습니다. 문서가 대학 도서관에 있는 어떤 책이라면 아주 큰 특성 행렬이 만들어집니다(sparse 매개변수를 True로 설정해야 합니다).

## 5.4 누락된 클래스 값 대체하기
이상적인 해결은 머신러닝 분류 알고리즘을 훈련하여 누락된 값을 예측하는 것입니다. 일반적으로 k-최근접 이웃(KNN) 분류기를 사용합니다.

In [35]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# 범주형 특성을 가진 특성 행렬을 만듭니다.
X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1, -0.21, -1.19]])

# 범주형 특성에 누락된 값이 있는 특성 행렬을 만듭니다.
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                       [np.nan, -0.67, -0.22]])

# KNN 학습기를 훈련합니다.
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X[:,1:], X[:,0])

# 누락된 값의 클래스를 예측합니다.
imputed_values = trained_model.predict(X_with_nan[:,1:])

# 예측된 클래스와 원본 특성을 열로 합칩니다.
X_with_imputed = np.hstack((imputed_values.reshape(-1,1), X_with_nan[:,1:]))

# 두 특성 행렬을 연결합니다.
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

다른 방법은 누락된 값을 특성에서 가장 자주 등장하는 값으로 채우는 것입니다.

In [33]:
from sklearn.impute import SimpleImputer

# 두 개의 특성 행렬을 합칩니다.
X_complete = np.vstack((X_with_nan, X))

imputer = SimpleImputer(strategy='most_frequent')
imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

범주형 특성에 누락된 값이 있을 때 가장 좋은 방법은 머신러닝 알고리즘으로 누락된 값을 예측하는 것입니다. 누락된 값이 있는 특성을 타깃으로 하고 다른 특성을 특성 행렬로 사용할 수 있습니다. 많이 사용하는 알고리즘은 KNN으로, k 최근접 이웃의 다수 클래스를 누락된 값에 할당합니다.

또는 특성에서 가장 자주 등장하는 클래스를 누락된 값으로 예측할 수 있습니다. KNN보다 덜 정교하지만 대규모 데이터에 적용하기 훨씬 쉽습니다. 두 경우 모두 대체된 값이 있는 샘플인지 나타내는 이진 특성을 추가하는 것이 좋습니다.

## 5.5 불균형한 클래스 다루기

더 많은 데이터를 모으세요. 이것이 불가능하면 모델 평가 지표를 바꾸세요. 잘 동작하지 않으면 (가능하다면) 모델에 내장된 클래스 가중치 매개변수를 사용하거나 다운샘플링이나 업샘플링을 고려해보세요. 이후 장에서 평가 지표를 다루겠습니다. 여기에서는 클래스 가중치 매개변수, 다운샘플링, 업샘플링에 초점을 맞춥니다. 

예제를 위해 클래스가 불균형한 데이터를 준비하겠습니다. 피셔의 붓꽃 데이터셋은 붓꽆의 품종을 나타내는 세 개 클래스(Iris setosa, Iris virginica, Iris versicolor)의 샘플을 50개씩 고르게 가지고 있습니다. 이때 불균형한 데이터셋을 만들기 위해서 Iris setosa 샘플 50개 중 40개를 삭제하겠습니다. 그다음 Iris virginica와 Iris versicolor 클래스를 합치겠습니다. 결과적으로 Iris setosa 샘플인지 아닌지를 가리키는 이진 타깃 벡터를 얻습니다. 여기에는 Iris setosa 샘플(클래스 0) 10개와 Iris setosa가 아닌 샘플(클래스 1) 90개가 있습니다.

In [36]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

iris = load_iris()

features = iris.data

target = iris.target

features = features[40:,:]
target = target[40:]

target = np.where((target == 0), 0, 1)

target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

사이킷런에 있는 많은 알고리즘은 훈련할 때 불균형한 영향을 줄일 수 있도록 클래스에 가중치를 부여할 수 있는 매개변수를 제공합니다. RandomForestClassifier는 class_weight 매개변수로 원하는 클래스 가중치를 직접 지정할 수 있습니다.

In [42]:
weights = {0: .9, 1: 0.1}

RandomForestClassifier(class_weight=weights)

RandomForestClassifier(class_weight={0: 0.9, 1: 0.1})

또는 balanced로 지정하여 클래스 빈도에 반비례하게 자동으로 가중치를 만들 수 있습니다.

In [43]:
RandomForestClassifier(class_weight="balanced")

RandomForestClassifier(class_weight='balanced')

다운샘플링에서는 다수 클래스에서 중복을 허용하지 않고 랜덤하게 샘플을 선택하여 소수 클래스와 같은 크기의 샘플 부분집합을 만듭니다. 예를 들면, 소수 클래스에 10개의 샘플이 있다면 다수 클래스에서 10개의 샘플을 랜덤하게 선택하여 총 20개의 샘플을 데이터로 사용합니다. 불균형하게 만든 붓꽃 데이터에 적용해보겠습니다.

In [45]:
# 각 클래스의 샘플 인덱스를 추춯합니다.
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]

# 각 클래스의 샘플 개수
n_class0 = len(i_class0)
n_class1 = len(i_class1)

# 클래스 0의 샘플만큼 클래스 1에서 중복을 허용하지 않고 랜덤하게 샘플을 뽑습니다.
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)

# 클래스 0의 타깃 벡터와 다운샘플링된 클래스 1의 타깃 벡터를 합칩니다.
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [46]:
# 클래스 0의 특성 행렬과 다운샘플링된 클래스 1의 특성 행렬을 합칩니다.
np.vstack((features[i_class0,:], features[i_class1_downsampled,:]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

또 다른 방법은 소수 클래스를 업샘플링하는 것입니다. 업샘플링은 다운샘플링과 반대 방식으로 매우 비슷하게 구현됩니다.

In [47]:
# 클래스 1의 샘플 개수만큼 클래스 0에서 중복을 허용하여 랜덤하게 샘플을 선택합니다.
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

# 클래스 0의 업샘플링된 타깃 벡터와 클래스 1의 타깃 벡터를 합칩니다.
np.concatenate((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [48]:
# 클래스 0의 업샘플링된 특성 행렬과 클래스 1의 특성 행렬을 합칩니다.
np.vstack((features[i_class0_upsampled,:], features[i_class1,:]))[0:5]

array([[4.4, 3.2, 1.3, 0.2],
       [4.5, 2.3, 1.3, 0.3],
       [4.8, 3. , 1.4, 0.3],
       [4.8, 3. , 1.4, 0.3],
       [5. , 3.5, 1.6, 0.6]])

1. 소수 클래스의 샘플을 더 많이 모읍니다. 하지만 이것이 불가능한 경우가 많기 때문에 다른 선택 사항을 고려해야 합니다.

2. 불균형한 클래스에 잘 맞는 모델 평가 지표를 사용합니다. 정확도는 모델 성능을 평가하는 데 자주 사용되는 지표입니다. 하지만 클래스가 불균형할 때 정확도는 잘 맞지 않습니다. 이후 장에서 소개할 더 나은 지표로는 오차 행렬, 정밀도, 재현율, F1 점수, ROC 곡선이 있습니다.

3. 일부 모델에서 제공하는 클래스 가중치 매개변수를 사용합니다. 이를 통해 알고리즘이 불균형한 클래스를 조정할 수 있습니다. 다행히 사이킷런에 잇는 많은 분류기들은 이에 적합한 class_weight 매개변수를 가지고 있습니다.

4. 다운샘플링/업샘플링을 사용합니다.다운샘플링에서 소수 클래스 크기와 동일하게 다수 클래스의 랜덤한 부분집합을 만듭니다. 샘플링에서는 다수 클래스 크기와 동일하게 소수 클래스로부터 중복을 허용하여 반복적으로 샘플을 뽑습니다. 다운샘플링과 업샘플링 중 어떤 것을 사용할지 여부는 문제에 따라 다릅니다. 일반적으로 두 전략을 모두 시도해보고 더 나은 결과를 내는 것을 선택합니다.